必要なライブラリーを取得

In [ ]:
# https://www.philschmid.de/fine-tune-flan-t5-peft
# install Hugging Face Libraries
!pip install "peft==0.2.0"
!pip install "transformers==4.29.0" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr

Googleドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 任意のディレクトリに移動
cd /content/drive/MyDrive/go/to/your/directory

モデルのロード

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "bigscience/bloomz-1b1"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype="auto",
    device_map="auto")

if torch.cuda.is_available():
    model = model.to("cuda")


データセットの準備

In [ ]:
import json

harry = 'Harry'
data = []

with open('en_train_set.json', 'r', encoding='utf-8') as f:
    d = json.load(f)

for j in range(50): # 160
  j += 1;
  session = "Session-" + str(j)
  dialogs = d[session]['dialogue']

  for i in range(len(dialogs) - 1):
    if i == range(len(dialogs) - 2):
      continue

    speaker = dialogs[i].split(':')[0]
    dialog = dialogs[i].split(':')[1]

    nextSpeaker = dialogs[i + 1].split(':')[0]
    nextDialog = dialogs[i + 1].split(':')[1]

    if speaker != harry and nextSpeaker == harry:
      data.append({
        'input': dialog,
        'output': nextDialog
      })

  # 会話シーンの背景情報
  scene = d[session]['scene']
  fixed_input = 'Tell me the background information of the Harry Potter.'
  data.append({
        'input': fixed_input,
        'output': scene
      })

print(data)
print(len(data))


学習データの準備

In [15]:
# プロンプトテンプレートの準備
def generate_prompt(data_point):
    result = f"""### 指示:
{data_point["input"]}

### 回答:
{data_point["output"]}
"""
    # 改行→<NL>
    result = result.replace('\n', '<NL>')
    return result

Tokenize

In [ ]:
CUTOFF_LEN = 256  # コンテキスト長

# トークナイズ
def tokenize(prompt, tokenizer):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

# トークナイズの動作確認
tokenize("hi there", tokenizer)

In [18]:
# データセットの準備
VAL_SET_SIZE = 1000

train_dataset = [] # 学習データ
val_dataset = [] # 検証データ

for i in range(len(data)):
  if i % 5 == 0:
      x = tokenize(generate_prompt(data[i]), tokenizer)
      val_dataset.append(x)
  else:
      x = tokenize(generate_prompt(data[i]), tokenizer)
      train_dataset.append(x)

LoRAモデルの準備

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# LoRAのパラメータ
lora_config = LoraConfig(
    r= 8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# モデルの前処理
model = prepare_model_for_int8_training(model)

# LoRAモデルの準備
model = get_peft_model(model, lora_config)

# 学習可能パラメータの確認
model.print_trainable_parameters()

ファインチューニング

In [ ]:
from torch.cuda import amp
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

# モデルをGPU上で単精度浮動小数点数に変換
model = model.float()

output_dir="./results/output_results"

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="no",
    report_to="tensorboard",
    evaluation_strategy="steps",
    eval_steps=200,
    save_total_limit=3,
    push_to_hub=False
)

# トレーナーの準備
trainer = Trainer(
    model=model, # 対象のモデル
    args=training_args, # 学習時の設定
    data_collator=data_collator, # データコレーター
    train_dataset=train_dataset, # 学習データセット
    eval_dataset=val_dataset, # 訓練データセット
)
model.config.use_cache = False

# 学習の実行
trainer.train() # 約2分で学習完了

# 学習済みモデルの保存

In [ ]:
peft_model_id="lora_results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

# モデルの検証

# 学習モデルの読み込み

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "lora_results"
config = PeftConfig.from_pretrained(peft_model_id)

# ベースモデルの準備
base_model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	load_in_8bit=True,
	device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# LoRAモデルのロード
model = PeftModel.from_pretrained(
	base_model,
	peft_model_id,
	state_dict=base_model.state_dict(),
	device_map="auto",
	torch_dtype=torch.float16)

# 評価モード
model.eval()

# プロンプトテンプレートの準備

In [23]:
# プロンプトテンプレートの準備
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"""### 指示:
{data_point["instruction"]}

### 入力:
{data_point["input"]}

### 回答:
"""
    else:
        result = f"""### 指示:
{data_point["instruction"]}

### 回答:
"""

    # 改行→<NL>
    result = result.replace('\n', '<NL>')
    return result

# テキスト生成関数の定義

In [24]:
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=256):
    # 推論
    prompt = generate_prompt({'instruction':instruction,'input':input})
    input_ids = tokenizer(prompt,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=False).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=maxTokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.75,
        top_k=40,
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()
    print(tokenizer.decode(outputs))

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # レスポンス内容のみ抽出
        sentinel = "### 回答:"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            result = decoded[sentinelLoc+len(sentinel):]
            print(result.replace("<NL>", "\n"))  # <NL>→改行
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
    else:
        print('Warning: no <eos> detected ignoring output')

学習後の結果確認

In [ ]:
# 中国語等で回答が返ってくる場合は、英語で回答するように記載してください。
generate("Tell me some secret of the Hogwarts School.")